Code for Paper


In [2]:
!pip install pandas matplotlib numpy
!pip install pandas chardet tqdm kneed plotly matplotlib scikit-learn sentence-transformers numpy tensorflow transformers torch
!pip install anthropic

In [15]:
import os 
import pandas as pd
pd.options.display.max_columns = None
from pathlib import Path
import xml.etree.ElementTree as ET
import re
import chardet
from tqdm import tqdm
from kneed import KneeLocator
import plotly.offline as py
import plotly.io as pio
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import init_notebook_mode, plot
from plotly.offline import init_notebook_mode, iplot
from matplotlib.colors import hsv_to_rgb
from IPython.display import display, Markdown
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from transformers import AutoTokenizer, AutoModel
import torch
import time
import requests
import json
from anthropic import Client
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score


Load the Summary Judgement dataset

In [6]:
df=pd.read_pickle('Corrected_df_backup2.pkl')

Setup the prompt for Claude Opus, saving responses 

In [ ]:
api_key="" 
headers = {
    'x-api-key': api_key,
    'anthropic-version': '2023-06-01',
    'content-type': 'application/json'
}


df.loc[:, 'primary_topic'] = None
df.loc[:, 'secondary_topic'] = None
df.loc[:, 'explanation'] = None
df.loc[:, 'confidence_score'] = None
legal_topics= [
    "Administrative and public law (PUB)" , "Advocacy (ADV)", "Agriculture (AGR)", "Animals (ANI)", "Aviation (AVI)", "Banking (BAN)", "Benefits and allowances (BEN)", "Business - general (BUS)", "Business - finance (BFI)", "Charities (CHA)", "Clinical negligence (CLI)", "Commercial – general (COG)", "Commercial – property (COP)", "Community care – general (CCG)", "Community care – legal aid (CCL)", "Company (COM)", "Competition (CMP)", "Construction (CON)", "Consumer – general (CSG)", "Consumer – financial services (CSF)", "Consumer – goods and services (CSG)", "Consumer – utilities (CUT)", "Contract (CTR)", "Conveyancing (CVE)", "Corporate finance (CFI)", "Crime – general (CRG)", "Crime – domestic violence (CRD)", "Crime – fraud (CRF)", "Crime – international (CRI)", "Crime – juvenile (CRJ)", "Crime – legal aid (CRL)", "Crime – sex (CRS)", "Cross-border (CRO)", "Data (DTA)", "Debt and bankruptcy – personal (DEB)", "Debt recovery (DTR)", "Defamation (DEF)", "Dispute resolution – arbitration (DAR)", "Dispute resolution – civil mediation (DRC)", "Dispute resolution – commercial mediation (DRO)", "Education (EDU)", "Elderly care – general (ECG)", "Elderly care – care home fees (ELH)", "Employment (EMP)", "Energy and natural resources (ENE)", "Environment (ENV)", "European Union law (EUL)", "Family – general (FAM)", "Family – children (FCH)", "Family – divorce and separation (FDS)", "Family – legal aid (FAL)", "Family – marriage and civil partnerships (FMC)", "Family – mediation (FME)", "Family – mediation – legal aid (FML)", "Family – public law – legal aid (FPL)", "Financial services (FIS)", "Gaming and betting (GAB)", "Health (HLT)", "Human rights (HRI)", "Immigration – general (IMG)", "Immigration – asylum (IMA)", "Immigration – legal aid (IML)", "Immigration – nationality and citizenship (IMN)", "Information Technology (ITE)", "Insolvency and restructuring – business (INS)", "Insurance (IUR)", "Intellectual property (IPR)", "International (INT)", "Judicial review – general (JRW)", "Judicial review – legal aid (JRL)", "Landlord and tenant – general (LRG)", "Landlord and tenant – commercial (LCO)", "Landlord and tenant – leasehold valuation (LLV)", "Landlord and tenant – residential (LRE)", "Lasting power of attorney (POA)", "Litigation – general (LIG)", "Litigation – small claims (LIS)", "Litigation – tickets and fines (LIT)", "Maritime and shipping (MAR)", "Media entertainment and sport (MED)", "Mental health – general (MHE)", "Mental health – legal aid (MHL)", "Mergers and acquisitions (MAA)", "Military (MIL)", "Mining (MNG)", "Motoring (MOT)", "Neighbour disputes (NDI)", "Pensions (PEN)", "Personal injury and damage – general (PIG)", "Personal injury and damage– road traffic accident (PIR)", "Planning (PLA)", "Police (POL)", "Prison law (PRZ)", "Private client – general (PCG)", "Private client – international (PCI)", "Private client – succession (PCS)", "Private client – trusts (PCT)", "Private client – wills (PCW)", "Procurement (PRO)", "Professional negligence (PNE)", "Tax – general (TAG)", "Tax – company and commercial (TAC)", "Tax – customs (TCU)", "Tax – employment (TAE)", "Tax – money and debt (TAM)", "Tax – personal (TAP)", "Travel and tourism (TAT)"
]
max_words= 80000
total_cost = 0 
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    case = row['content']
    word_count = len(case.split())
    print(f"Processing row {index + 1}")
    
    input_cost = (word_count / 1000000) * 15
    total_cost += input_cost   
    print(f"Input cost: ${input_cost:.2f}, Cumulative cost: ${total_cost:.2f}")

    if word_count > max_words:
        print("Case exceeds maximum tokens. Skipping.")
        continue

    full_prompt = f"""
Dear Claude Opus,

I would like you to carefully review the details of a legal case that has been filed by a judge. After analyzing the facts and legal issues involved, please determine the most appropriate primary topic that the case falls under, based only on the following comprehensive list of legal topics:

{", ".join(legal_topics)}

If the case involves multiple legal issues, please identify one primary topic that is most central and relevant to the case. Avoid selecting a topic that is only tangentially related or a peripheral issue. The topic description should provide sufficient information for a judge to properly classify the case.

After determining the primary topic, please consider whether the case can also be categorized under any secondary topics from the legal topics list above only. If so, include the most relevant secondary topic. If there are no applicable secondary topics, simply state 'none'.

Before providing your response, please ensure that the selected primary and secondary topics are actually present in the list of legal topics provided above. If the topics you have chosen are not found in the list, please reconsider your selection and choose topics that are included in the list that meet the description of the primary and secondary topic. 

Please format your response as follows:
Primary topic: [topic 1]
Secondary topic: [topic] or none

In your response, please provide a brief explanation (around 10 words) of your reasoning and cite any relevant legal principles or precedents that support your decision. Use the format:[Explanation: ]. Additionally, include a confidence score (1-5, with 5 being the most confident) for your topic allocation. Use the format:[Confidence score: ]

Thank you for your thorough analysis and attention to detail in this matter.


Case details:
{case}
"""
    data = json.dumps({
        "model": "claude-3-opus-20240229",
        "max_tokens": 1024,
        "temperature": 0,
        "messages": [{"role": "user", "content": full_prompt}]
    })
    
    response = requests.post('https://api.anthropic.com/v1/messages', headers=headers, data=data)
    if response.status_code == 200:
        response_data = response.json()
        content_text = response_data['content'][0]['text']
        
        primary_topics = content_text.split("Primary topic: ")[1].split("\n")[0]
        secondary_topic = content_text.split("Secondary topic: ")[1].split("\n")[0]
        explanation = content_text.split("Explanation: ")[1].split("\n")[0]
        confidence_score = content_text.split("Confidence score: ")[1].split("\n")[0]
        
        df.loc[index, 'primary_topic'] = primary_topics
        df.loc[index, 'secondary_topic'] = secondary_topic
        df.loc[index, 'explanation'] = explanation
        df.loc[index, 'confidence_score'] = confidence_score
        
        print(f"Stored results for row {index + 1}")
        
        display(Markdown(f"### Results for Row {index + 1}"))
        display(Markdown(f"**Primary Topics:** {primary_topics}"))
        display(Markdown(f"**Secondary Topic:** {secondary_topic}"))
        display(Markdown(f"**Explanation:** {explanation}"))
        display(Markdown(f"**Confidence Score:** {confidence_score}"))
         
    else:
        print(f"Failed to process row {index + 1} due to API error: {response.status_code}, Response Content: {response.text}")


    time.sleep(1)  

df.to_pickle('backup TM Claude.pkl')

Randomly sample 342 cases to check Claude's primary and secondary topic evaluation manually 

In [ ]:
sample_df = df.sample(n=342, random_state=1)

In [ ]:
sample_df.loc[:, 'evaluation'] = None
sample_df.to_pickle('Sample from TM Claude.pkl')

Detect hallucinations by comparing the primary topics labels with the legal taxonomy given to Claude in the prompt


In [ ]:
print(df['primary_topic'].unique())

Consider which of these topics is not present in the legal taxonomy. 

Move the cases which have hallucinations to a sepparate dataframe for manual processing

After manually correcting these cases reinsert them back into the dataframe

    

In [27]:
df_corrected=pd.read_pickle('df_hal_re_run.pkl')

In [ ]:
checked_df = pd.concat([df_no_hal, df_corrected], axis=0, ignore_index=True)

In [54]:
def normalize_text(text):
    import re

    text = text.lower().strip()

    text = re.sub(r'[^\w\s]', '', text)

    text = re.sub(r'\s+', ' ', text)
    return text

def find_unmatched_topics(df, legal_topics):

    df_topics = set(normalize_text(topic) for topic in df['primary_topic'].unique())

    legal_topics_set = set(normalize_text(topic) for topic in legal_topics)

    unmatched_topics = df_topics.difference(legal_topics_set)
    

    if unmatched_topics:
        print("Unmatched Topics (Not in Provided List):")
        for topic in sorted(unmatched_topics):
            print(topic)
        print(f"\nTotal Unmatched Topics: {len(unmatched_topics)}")
    else:
        print("All topics in the DataFrame match the provided legal topics list after normalization.")


find_unmatched_topics(df, legal_topics)


All topics in the DataFrame match the provided legal topics list after normalization.


In [32]:
checked_df.to_pickle('checked_df.pkl')

Insert a new column that allocates each topic within the broader areas of law, and an 'x' if it is not allocated

In [36]:
topic_to_area = {
    "PUB": "Public law", "ENV": "Public law", "MIL": "Public law", "MNG": "Public law",
    "MOT": "Public law", "PLA": "Public law", "POL": "Public law", "TAG": "Public law",
    "TAC": "Public law", "TCU": "Public law", "TAE": "Public law", "TAM": "Public law",
    "TAP": "Public law", "ADV": "Dispute resolution law", "DAR": "Dispute resolution law",
    "DRC": "Dispute resolution law", "DRO": "Dispute resolution law", "JRW": "Dispute resolution law",
    "JRL": "Dispute resolution law", "LIG": "Dispute resolution law", "LIS": "Dispute resolution law",
    "LIT": "Dispute resolution law", "AGR": "Commercial law", "ANI": "Commercial law",
    "AVI": "Commercial law", "BAN": "Commercial law", "BUS": "Commercial law",
    "BFI": "Commercial law", "COG": "Commercial law", "COP": "Commercial law",
    "COM": "Commercial law", "CMP": "Commercial law", "CON": "Commercial law",
    "CTR": "Commercial law", "CVE": "Commercial law", "CFI": "Commercial law",
    "DTR": "Commercial law", "ENE": "Commercial law", "FIS": "Commercial law",
    "GAB": "Commercial law", "ITE": "Commercial law", "INS": "Commercial law",
    "IUR": "Commercial law", "IPR": "Commercial law", "LCO": "Commercial law",
    "MAR": "Commercial law", "MED": "Commercial law", "MAA": "Commercial law",
    "PRO": "Commercial law", "PNE": "Commercial law", "BEN": "Personal and consumer matters",
    "CHA": "Personal and consumer matters", "CLI": "Personal and consumer matters",
    "CCG": "Personal and consumer matters", "CCL": "Personal and consumer matters",
    "CSG": "Personal and consumer matters", "CSF": "Personal and consumer matters",
    "CUT": "Personal and consumer matters", "DTA": "Personal and consumer matters",
    "DEB": "Personal and consumer matters", "DEF": "Personal and consumer matters",
    "EDU": "Personal and consumer matters", "ECG": "Personal and consumer matters",
    "ELH": "Personal and consumer matters", "EMP": "Personal and consumer matters",
    "FAM": "Personal and consumer matters", "FCH": "Personal and consumer matters",
    "FDS": "Personal and consumer matters", "FAL": "Personal and consumer matters",
    "FMC": "Personal and consumer matters", "FME": "Personal and consumer matters",
    "FML": "Personal and consumer matters", "FPL": "Personal and consumer matters",
    "HLT": "Personal and consumer matters", "HRI": "Personal and consumer matters",
    "LRG": "Personal and consumer matters", "LLV": "Personal and consumer matters",
    "LRE": "Personal and consumer matters", "POA": "Personal and consumer matters",
    "MHE": "Personal and consumer matters", "MHL": "Personal and consumer matters",
    "NDI": "Personal and consumer matters", "PEN": "Personal and consumer matters",
    "PIG": "Personal and consumer matters", "PIR": "Personal and consumer matters",
    "PCG": "Personal and consumer matters", "PCI": "Personal and consumer matters",
    "PCS": "Personal and consumer matters", "PCT": "Personal and consumer matters",
    "PCW": "Personal and consumer matters", "TAT": "Personal and consumer matters",
    "CRG": "Criminal law", "CRD": "Criminal law", "CRF": "Criminal law", "CRI": "Criminal law",
    "CRJ": "Criminal law", "CRL": "Criminal law", "CRS": "Criminal law", "PRZ": "Criminal law",
    "CRO": "International law", "EUL": "International law", "IMG": "International law",
    "IMA": "International law", "IML": "International law", "IMN": "International law",
    "INT": "International law"
}

import re
def get_area(topic):
    match = re.search(r'\b[A-Z]{3}\b', topic)
    abbr = match.group() if match else 'x'
    return topic_to_area.get(abbr, 'x')
checked_df['primary_area'] = checked_df['primary_topic'].apply(get_area)
df=checked_df.copy()

In [61]:
print(df['primary_area'].unique())                        

['Personal and consumer matters' 'Dispute resolution law' 'Commercial law'
 'Criminal law' 'International law' 'Public law']


In [7]:
pio.renderers.default = 'browser'  

df_pivot = df.pivot_table(index='primary_area', aggfunc='size')
df_pivot = df_pivot.reset_index().rename(columns={0: 'Number of Cases'})


hues = np.linspace(0, 1, len(df_pivot), endpoint=False)  
colors = [hsv_to_rgb([h, 0.2, 0.95]) for h in hues]  #


fig1 = go.Figure(data=[go.Bar(
    x=df_pivot['primary_area'],
    y=df_pivot['Number of Cases'],
    text=df_pivot['Number of Cases'],
    textposition='auto',
    marker=dict(
        color=['rgb({},{},{})'.format(int(r*255), int(g*255), int(b*255)) for r, g, b in colors],  # Convert to RGB color format for CSS
        line=dict(color='#333333', width=1)
    )
)])

fig1.update_layout(
    title_text='Number of Cases Across Higher-level Areas of Law',
    xaxis_title='Higher-level Area',
    yaxis_title='Number of Cases',
    font=dict(size=32, family='Arial, sans-serif'),  
    plot_bgcolor='white',   
    paper_bgcolor='white'  
)

pio.show(fig1)


In [13]:
from plotly.offline import init_notebook_mode, plot
import plotly.graph_objects as go
import pandas as pd
import numpy as np
init_notebook_mode(connected=True)

def plot_primary_topic_counts(df, area):

    filtered_df = df[df['primary_area'] == area]
    
   
    topic_count = filtered_df.groupby('primary_topic').size().reset_index(name='Number of Cases')

    colors = ['rgb({},{},{})'.format(int(255 * (0.6 + 0.4 * np.random.rand())),
                                     int(255 * (0.6 + 0.4 * np.random.rand())),
                                     int(255 * (0.6 + 0.4 * np.random.rand()))) for _ in range(len(topic_count['primary_topic']))]
    
  
    fig = go.Figure(data=[go.Bar(
        x=topic_count['primary_topic'],
        y=topic_count['Number of Cases'],
        marker=dict(color=colors, line=dict(color='#333333', width=1)),
        text=topic_count['Number of Cases'],
        textposition='auto'
    )])


    fig.update_layout(
        title_text=f'Number of Cases per Specific Area of {area}',
        xaxis_title='Area of law ',
        yaxis_title='Number of Cases',
        template='plotly_white',
        font=dict(size=32) 
    )


    plot(fig, auto_open=True)


areas = df['primary_area'].unique()


for area in areas:
    plot_primary_topic_counts(df, area)


In [ ]:
area_topic_count = df.groupby('primary_area')['primary_topic'].nunique().reset_index(name='Number of Unique Topics')
print(area_topic_count)

In [ ]:
pio.renderers.default = 'notebook'

def plot_primary_area_changes_over_time(df, pastel=False):

    df['formatted_date'] = pd.to_datetime(df['fo_date'])
    

    monthly_area_data = df.groupby([df['formatted_date'].dt.to_period("M"), 'primary_area']).size().reset_index(name='Cases')
    monthly_area_data['formatted_date'] = monthly_area_data['formatted_date'].dt.to_timestamp()  

    pivot_data = monthly_area_data.pivot(index='formatted_date', columns='primary_area', values='Cases').fillna(0)


    primary_areas = pivot_data.columns
    hues = np.linspace(0, 1, len(primary_areas), endpoint=False)
    if pastel:
        colors = [hsv_to_rgb([h, 0.2, 0.95]) for h in hues] 
    else:
        colors = [hsv_to_rgb([h, 0.5, 0.95]) for h in hues] 
    color_map = {area: 'rgb({},{},{})'.format(int(r*255), int(g*255), int(b*255)) for area, (r, g, b) in zip(primary_areas, colors)}

    fig = px.line(pivot_data, title='Changes in Higher-level Area of Law Over Time', color_discrete_map=color_map)
    fig.update_xaxes(dtick="M12", tickformat="%Y")  
    fig.update_layout(xaxis_title='Year', yaxis_title='Number of Cases', legend_title='Higher-level Area',
                      plot_bgcolor='white', paper_bgcolor='white',
                      xaxis=dict(showgrid=True, gridcolor='lightgrey'),
                      yaxis=dict(showgrid=True, gridcolor='lightgrey'))
    fig.show()


plot_primary_area_changes_over_time(df, pastel=False)
pio.renderers.default = 'browser'  

df_pivot = df.pivot_table(index='primary_area', aggfunc='size')
df_pivot = df_pivot.reset_index().rename(columns={0: 'Number of Cases'})


hues = np.linspace(0, 1, len(df_pivot), endpoint=False)  
colors = [hsv_to_rgb([h, 0.2, 0.95]) for h in hues]  #


fig1 = go.Figure(data=[go.Bar(
    x=df_pivot['primary_area'],
    y=df_pivot['Number of Cases'],
    text=df_pivot['Number of Cases'],
    textposition='auto',
    marker=dict(
        color=['rgb({},{},{})'.format(int(r*255), int(g*255), int(b*255)) for r, g, b in colors],  # Convert to RGB color format for CSS
        line=dict(color='#333333', width=1)
    )
)])

fig1.update_layout(
    title_text='Number of Cases Across Higher-level Areas of Law',
    xaxis_title='Higher-level Area',
    yaxis_title='Number of Cases',
    font=dict(size=32, family='Arial, sans-serif'),  
    plot_bgcolor='white',  
    paper_bgcolor='white'  
)

pio.show(fig1)


In [ ]:
def generate_pastel_colors(n):
    return ['rgb({},{},{})'.format(int(255 * (0.6 + 0.4 * np.random.rand())),
                                   int(255 * (0.6 + 0.4 * np.random.rand())),
                                   int(255 * (0.6 + 0.4 * np.random.rand()))) for _ in range(n)]

def plot_cases_per_tier(df):

    tier_summary = df.groupby('tier').size().reset_index(name='Number of Cases')
    tier_summary = tier_summary.sort_values(by='Number of Cases', ascending=False)  

    colors = generate_pastel_colors(len(tier_summary['tier']))

    fig = go.Figure(data=[go.Bar(
        x=tier_summary['tier'],
        y=tier_summary['Number of Cases'],
        marker=dict(color=colors),  
        text=tier_summary['Number of Cases'],
        textposition='auto'
    )])

    fig.update_layout(
        title_text='Number of Cases per Tier',
        xaxis_title='Tier',
        yaxis_title='Number of Cases',
        template='plotly_white'
    )


    fig.show()


plot_cases_per_tier(df)


In [16]:
def plot_stacked_bar_chart(df):
   
    df['formatted_date'] = pd.to_datetime(df['fo_date'])
    

    monthly_area_data = df.groupby([df['formatted_date'].dt.to_period("M"), 'primary_area']).size().reset_index(name='Cases')
    monthly_area_data['formatted_date'] = monthly_area_data['formatted_date'].dt.to_timestamp() 

    fig = px.bar(monthly_area_data, x='formatted_date', y='Cases', color='primary_area', 
                 title='Monthly Case Distribution by Higher-level Area of Law', 
                 labels={'formatted_date': 'Date', 'Cases': 'Number of Cases'},
                 text='Cases')
    fig.update_xaxes(dtick="M12", tickformat="%Y")  
    fig.update_layout(
        barmode='stack', 
        xaxis_title='Year', 
        yaxis_title='Higher-level Area of Law', 
        legend_title='Higher-level Area of Law',
        font=dict(size=32),
        plot_bgcolor='white', 
        paper_bgcolor='white'  
    )

   
    plot(fig, auto_open=True)

plot_stacked_bar_chart(df)


In [25]:
def plot_cumulative_stacked_bar_chart(df):
    df['formatted_date'] = pd.to_datetime(df['fo_date'])
    yearly_area_data = df.groupby([df['formatted_date'].dt.to_period("Y"), 'primary_area']).size().reset_index(name='Cases')
    yearly_area_data['formatted_date'] = yearly_area_data['formatted_date'].dt.to_timestamp()
    yearly_area_data['Cumulative Cases'] = yearly_area_data.groupby('primary_area')['Cases'].cumsum()
    

    pastel_palette = px.colors.qualitative.Pastel

    fig = px.bar(yearly_area_data, x='formatted_date', y='Cumulative Cases', color='primary_area', 
                 title='Cumulative Case Distribution by Higher-level Areas of Law',
                 labels={'formatted_date': 'Year', 'Cumulative Cases': 'Cumulative Number of Cases'},
                 text='Cumulative Cases', color_discrete_sequence=pastel_palette)
    fig.update_xaxes(dtick="M12", tickformat="%Y")
    fig.update_layout(
        barmode='stack',
        xaxis_title='Year',
        yaxis_title='Cumulative Number of Cases',
        legend_title='Higher-level Areas of Law',
        title_font_size=32,
        font=dict(size=32),
        legend_title_font_size=32,
        xaxis_title_font_size=32,
        yaxis_title_font_size=32,
        plot_bgcolor='rgba(0,0,0,0)',  
        paper_bgcolor='rgba(0,0,0,0)'  
    )
    pyo.plot(fig, filename='cumulative_stacked_bar_chart.html')

plot_cumulative_stacked_bar_chart(df)


In [17]:
def plot_stacked_bar_chart(df):

    df['formatted_date'] = pd.to_datetime(df['fo_date'])

    monthly_area_data = df.groupby([df['formatted_date'].dt.to_period("M"), 'primary_area']).size().reset_index(name='Cases')
    monthly_area_data['formatted_date'] = monthly_area_data['formatted_date'].dt.to_timestamp()  

    fig = px.bar(monthly_area_data, x='formatted_date', y='Cases', color='primary_area', 
                 title='Monthly Case Distribution by Higher-level Area of Law', 
                 labels={'formatted_date': 'Date', 'Cases': 'Number of Cases'})
    fig.update_xaxes(dtick="M12", tickformat="%Y")  
    fig.update_layout(
        barmode='stack', 
        xaxis_title='Year', 
        yaxis_title='Higher-level Area of Law', 
        legend_title='Higher-level Area of Law',
        font=dict(size=32),
        plot_bgcolor='white',  
        paper_bgcolor='white'  
    )


    plot(fig, auto_open=True)

plot_stacked_bar_chart(df)


In [ ]:
topic_counts = df.groupby(['year', 'primary_topic']).size().reset_index(name='num_cases')

top_topics_per_year = topic_counts.sort_values(by=['year', 'num_cases'], ascending=[True, False])
top_topics_per_year = top_topics_per_year.groupby('year').head(2)

top_topic_list = top_topics_per_year['primary_topic'].unique()
filtered_topics = topic_counts[topic_counts['primary_topic'].isin(top_topic_list)]

filtered_topics = filtered_topics[filtered_topics['year'] != 1995]


bins = [0, 6, 11, 16, 21, 26, 32, float('inf')]
labels = ['1-5', '6-10', '11-15', '16-20', '21-25', '26-32', '>32']
filtered_topics['case_bin'] = pd.cut(filtered_topics['num_cases'], bins=bins, labels=labels, include_lowest=True)


solid_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

filtered_topics['normalized_cases'] = filtered_topics['num_cases'] / filtered_topics['num_cases'].max()

fig1 = px.scatter(filtered_topics, 
                  x='year', y='primary_topic', 
                  size='normalized_cases',
                  color='case_bin',
                  category_orders={"case_bin": labels, "primary_topic": sorted(filtered_topics['primary_topic'].unique())},
                  color_discrete_map=dict(zip(labels, solid_colors)),
                  title='Most Frequent Higher-level Area of Law by Year',
                  labels={'year': 'Year', 'primary_topic': 'Higher-level Area of Law', 'case_bin': 'Number of Cases'})

fig1.update_layout(
    legend_title_text='Number of Cases',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.1,
        xanchor="center",
        x=0.5
    ),
    plot_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickmode='linear',
        tick0=filtered_topics['year'].min(),
        dtick=1
    ),
    margin=dict(r=50)
)

fig1.show(config={'toImageButtonOptions': {'format': 'svg', 'filename': 'custom_filename', 'height': 600, 'width': 800, 'scale': 2}})

primary_topic_order = sorted(filtered_topics['primary_topic'].unique(), key=lambda x: (x == 'Contract (CTR)', x))

fig2 = px.scatter(filtered_topics, 
                  x='year', y='primary_topic', 
                  size='normalized_cases',
                  color='case_bin',
                  category_orders={"case_bin": labels, "primary_topic": primary_topic_order},
                  color_discrete_map=dict(zip(labels, solid_colors)),
                  title='Most Frequent Higher-level Area of Law by Year',
                  labels={'year': 'Year', 'primary_topic': 'Higher-level Area of Law', 'case_bin': 'Number of Cases'})

fig2.update_layout(
    legend_title_text='Number of Cases',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.1,
        xanchor="center",
        x=0.5
    ),
    plot_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickmode='linear',
        tick0=filtered_topics['year'].min(),
        dtick=1
    ),
    margin=dict(r=50)
)

fig2.show(config={'toImageButtonOptions': {'format': 'svg', 'filename': 'custom_filename', 'height': 600, 'width': 800, 'scale': 2}})


In [20]:
def plot_court_case_counts(df):
   
    court_count = df.groupby('court').size().reset_index(name='Number of Cases')


    court_count = court_count.sort_values('Number of Cases', ascending=False)
    
   
    colors = ['rgb({},{},{})'.format(int(255 * (0.6 + 0.4 * np.random.rand())),
                                     int(255 * (0.6 + 0.4 * np.random.rand())),
                                     int(255 * (0.6 + 0.4 * np.random.rand()))) for _ in range(len(court_count['court']))]
    

    fig = go.Figure(data=[go.Bar(
        x=court_count['Number of Cases'],
        y=court_count['court'],
        marker=dict(color=colors, line=dict(color='#333333', width=1)),
        text=court_count['Number of Cases'],
        textposition='auto',
        orientation='h'
    )])

    fig.update_layout(
        title_text='Number of Cases per Court',
        xaxis_title='Number of Cases',
        yaxis_title='Court',
        template='plotly_white',
        font=dict(size=32),
        plot_bgcolor='white',  
        paper_bgcolor='white'  
    )


    plot(fig, auto_open=True)


plot_court_case_counts(df)


In [26]:
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot

def plot_court_case_counts(df):
 
    df['court'] = df['court'].replace({
        'England and Wales High Court (Queen\'s Bench Division)': 'England and Wales High Court (Queen\'s and King\'s Bench Division)',
        'England and Wales High Court (King\'s Bench Division)': 'England and Wales High Court (Queen\'s and King\'s Bench Division)'
    })
    
   
    court_count = df.groupby('court').size().reset_index(name='Number of Cases')
 
    court_count = court_count.sort_values('Number of Cases', ascending=False)
 
    colors = ['rgb({},{},{})'.format(int(255 * (0.6 + 0.4 * np.random.rand())),
                                     int(255 * (0.6 + 0.4 * np.random.rand())),
                                     int(255 * (0.6 + 0.4 * np.random.rand()))) for _ in range(len(court_count['court']))]
 
    fig = go.Figure(data=[go.Bar(
        x=court_count['Number of Cases'],
        y=court_count['court'],
        marker=dict(color=colors, line=dict(color='#333333', width=1)),
        text=court_count['Number of Cases'],
        textposition='auto',
        orientation='h'
    )])
    
 
    fig.update_layout(
        title_text='Number of Cases per Court',
        xaxis_title='Number of Cases',
        yaxis_title='Court',
        template='plotly_white',
        font=dict(size=32),
        plot_bgcolor='white',  
        paper_bgcolor='white'  
    )
    
 
    plot(fig, auto_open=True)

  
plot_court_case_counts(df)


In [28]:
def plot_primary_topic_distribution_per_court(df):
    topic_data = df.groupby(['court', 'primary_topic']).size().reset_index(name='Count')
    top_courts = topic_data.groupby('court')['Count'].sum().nlargest(20).index  # Get the top 6 courts by total cases

  
    topic_data = topic_data[topic_data['court'].isin(top_courts)]


    colors = ['rgb({},{},{})'.format(int(255 * (0.6 + 0.4 * np.random.rand())),
                                     int(255 * (0.6 + 0.4 * np.random.rand())),
                                     int(255 * (0.6 + 0.4 * np.random.rand()))) for _ in range(len(topic_data['primary_topic'].unique()))]
    

    for court in top_courts:
        court_data = topic_data[topic_data['court'] == court]
        fig = px.bar(court_data, x='primary_topic', y='Count', color='primary_topic',
                     title=f'Specific Areas of Law Distribution for {court}',
                     labels={'primary_topic': 'Higher-level Area of Law', 'Count': 'Number of Cases'},
                     color_discrete_sequence=colors)
        fig.update_traces(marker=dict(line=dict(color='#333333', width=1)), textposition='outside', texttemplate='%{y}')
        fig.update_layout(
            xaxis_title='Specific Area of Law',
            yaxis_title='Number of Cases',
            font=dict(size=32),
            plot_bgcolor='rgba(0,0,0,0)',  
            paper_bgcolor='rgba(0,0,0,0)',  
            showlegend=False 
        )
        pyo.plot(fig, filename=f'primary_topic_distribution_{court}.html')

plot_primary_topic_distribution_per_court(df)
